In [1]:
%matplotlib
import glob
import random
import itertools
import math
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score
from python_speech_features import mfcc, fbank, logfbank
from pydub import AudioSegment
from collections import Counter
from multiprocessing import Pool
from tqdm import tqdm

Using matplotlib backend: Qt5Agg


In [2]:
reciters_paths = glob.glob('./reciters/*/*')

train_paths_dictionary = {}
validation_paths_dictionary = {}
test_paths_dictionary = {}
all_paths_dictionary = {}

for i, j in itertools.groupby(reciters_paths, key=lambda x: x.split('/')[-2]):
    files = list(j)
    all_paths_dictionary[int(i)] = files
    train_paths, test_paths = train_test_split(files, test_size=0.30, random_state=13)
    test_paths, validation_paths = train_test_split(test_paths, test_size=0.30, random_state=13)
    train_paths_dictionary[int(i)] = train_paths
    test_paths_dictionary[int(i)] = test_paths
    validation_paths_dictionary[int(i)] = validation_paths

In [3]:
random.seed(a=13)

In [4]:
original_distribution = []
for i, j in itertools.groupby(reciters_paths, key=lambda x: x.split('/')[-2]):
    original_distribution += [int(i)] * len(list(j))
sns.distplot(original_distribution, bins=50, kde=False, rug=False, axlabel='reciter id')

In [5]:
original_distribution_counter = Counter(original_distribution)
sampled_distribution = []
for k in original_distribution_counter.keys():
    sampled_distribution += [k] * min(10, original_distribution_counter[k])
sns.distplot(sampled_distribution, bins=50, kde=False, rug=False, axlabel='reciter id')

In [6]:
original_lengths = []
sampled_lengths = []
for reciter_id, reciter_path in tqdm(all_paths_dictionary.items()):
    for clip_path in random.sample(reciter_path, min(10, len(reciter_path))):
        clip = AudioSegment.from_mp3(clip_path).split_to_mono()[0]
        original_lengths.append(int(len(clip)/1000))
        segments_in_clip = int(len(clip)/5000)
        sampled_lengths.append(min(5, segments_in_clip)*5)

100%|██████████| 50/50 [18:04<00:00, 21.68s/it]


In [15]:
sns.distplot(original_lengths, bins=50, kde=False, rug=False, axlabel='clip length')

In [16]:
sns.distplot(sampled_lengths, bins=50, kde=False, rug=False, axlabel='clip length')

In [29]:
np.median(original_lengths)/60

6.366666666666666